<a href="https://colab.research.google.com/github/rithikkulkarni/ALDA-Course-Project/blob/main/finbert_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initial preprocessing code

In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/rithikkulkarni/ALDA-Course-Project/refs/heads/main/datasets/stock_tweets.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80793 entries, 0 to 80792
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          80793 non-null  object
 1   Tweet         80793 non-null  object
 2   Stock Name    80793 non-null  object
 3   Company Name  80793 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


In [3]:
!pip install transformers

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

In [5]:
from transformers import pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert", device="cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [11]:
### Function for using FinBERT to label our data for supervised learning
def label_data(tweet_text):
  temp = pipe(tweet_text)
  if 'positive' in str(temp[0]):
    return 'positive'
  elif 'negative' in str(temp[0]):
    return 'negative'
  else:
    return 'neutral'

### Function for using FinBERT that should be faster because it process multiple tweets in parallel
def label_data_batch(tweet_texts, batch_size=32):
    results = pipe(tweet_texts, batch_size=batch_size)
    labels = []
    for res in results:
        if 'positive' in str(res):
            labels.append('positive')
        elif 'negative' in str(res):
            labels.append('negative')
        else:
            labels.append('neutral')
    return labels

In [14]:
# import numpy as np

# batch_size = 32  # Adjust based on memory and speed
# tweet_batches = np.array_split(df['Tweet'].to_numpy(), len(df) // batch_size + 1)
# label_batches = [label_data_batch(batch.tolist()) for batch in tweet_batches]

# # Flatten the list of label batches
# df['label'] = np.concatenate(label_batches)

import torch
from datasets import Dataset

# Ensure GPU is enabled
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("text-classification", model="ProsusAI/finbert", device=device)

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['Tweet']])  # Keep only relevant column

# Define a function to process tweets
def label_data_batch(batch):
    results = pipe(batch['Tweet'], batch_size=32)  # Adjust batch size
    return {'label': [res['label'].lower() for res in results]}  # Convert to lowercase

# Apply the function to the dataset
labeled_dataset = dataset.map(label_data_batch, batched=True)

# Convert back to DataFrame
df['label'] = labeled_dataset['label']

Device set to use cpu


Map:   0%|          | 0/80793 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
df.info()

In [10]:
df.rename(columns={'label': 'Label'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80793 entries, 0 to 80792
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          80793 non-null  object 
 1   Tweet         80793 non-null  object 
 2   Stock Name    80793 non-null  object 
 3   Company Name  80793 non-null  object 
 4   sentiment     80793 non-null  float64
dtypes: float64(1), object(4)
memory usage: 3.1+ MB


In [ ]:
df.Label.value_counts()

In [ ]:
### To be used for stock price

import yfinance as yf

ticker = yf.Ticker("AAPL")
hist = ticker.history(start="2024-02-05", end="2024-02-12")
print(hist)

In [8]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Download VADER lexicon if it's not already present
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Compute the continuous compound sentiment score for each tweet
df['sentiment'] = df['Tweet'].apply(lambda tweet: analyzer.polarity_scores(tweet)['compound'])

# Optional: Display the first few rows to verify the new column
print(df.head())


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                        Date  \
0  2022-09-29 23:41:16+00:00   
1  2022-09-29 23:24:43+00:00   
2  2022-09-29 23:18:08+00:00   
3  2022-09-29 22:40:07+00:00   
4  2022-09-29 22:27:05+00:00   

                                               Tweet Stock Name Company Name  \
0  Mainstream media has done an amazing job at br...       TSLA  Tesla, Inc.   
1  Tesla delivery estimates are at around 364k fr...       TSLA  Tesla, Inc.   
2  3/ Even if I include 63.0M unvested RSUs as of...       TSLA  Tesla, Inc.   
3  @RealDanODowd @WholeMarsBlog @Tesla Hahaha why...       TSLA  Tesla, Inc.   
4  @RealDanODowd @Tesla Stop trying to kill kids,...       TSLA  Tesla, Inc.   

   sentiment  
0     0.0772  
1     0.0000  
2     0.2960  
3    -0.7568  
4    -0.8750  


In [9]:
df.head(50)

,Date,Tweet,Stock Name,Company Name,sentiment
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",0.0772
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",0.0000
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",0.2960
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",-0.7568
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",-0.8750
5,2022-09-29 22:25:53+00:00,@RealDanODowd @Tesla This is you https://t.co/...,TSLA,"Tesla, Inc.",0.0000
6,2022-09-29 22:24:22+00:00,For years @WholeMarsBlog viciously silenced @T...,TSLA,"Tesla, Inc.",-0.9359
7,2022-09-29 22:23:54+00:00,$NIO just because I'm down money doesn't mean ...,TSLA,"Tesla, Inc.",0.2500
8,2022-09-29 22:23:28+00:00,50 likes for some $SPY $TSLA charts to study!\...,TSLA,"Tesla, Inc.",0.4753
9,2022-09-29 22:15:01+00:00,@MrJames__321 @KellyRoofing @TeslaSolar @elonm...,TSLA,"Tesla, Inc.",-0.8625


In [13]:
df.to_csv('continuous_labeled_tweets.csv', index=False)